In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

# Transformer Encoder for time series data
from src.models.transformer_ts_tf import Transformer

2024-07-15 23:57:34.760093: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-15 23:57:34.762387: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 23:57:34.813445: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 23:57:34.814415: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 23:57:35.793089: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

<IPython.core.display.Javascript object>

# Functions and definitions

### Convert train/test data to 3D format

In [3]:
def generate_sequences_helper(
    dataset, cement_types, dates=None, timesteps=None, split_by_cement_type=False
):
    index_train = dataset["y_train"].index
    index_test = dataset["y_test"].index

    dataset["y_train"] = dataset["y_train"].reset_index(drop=True)
    dataset["y_test"] = dataset["y_test"].reset_index(drop=True)

    if dates is not None:
        dataset["dates_train"] = dates[index_train].reset_index(drop=True)
        dataset["dates_test"] = dates[index_test].reset_index(drop=True)

    dataset["cement_types_train"] = cement_types.loc[index_train].reset_index(drop=True)
    dataset["cement_types_test"] = cement_types.loc[index_test].reset_index(drop=True)

    dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def generate_sequences(
    dataset,
    timesteps,
    split_by_cement_type=False,
    train_columns=[],
    test_columns=[],
):
    if split_by_cement_type:
        dataset["x_train"], dataset["y_train"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_train"],
                    pd.DataFrame(dataset["x_train"], columns=train_columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ),
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_test"],
                    pd.DataFrame(dataset["x_test"], columns=test_columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ),
            timesteps,
        )
    else:
        dataset["x_train"], dataset["y_train"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_train"], columns=train_columns),
                    dataset["y_train"],
                ],
                axis=1,
            ).values,
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_test"], columns=test_columns),
                    dataset["y_test"],
                ],
                axis=1,
            ).values,
            timesteps,
        )
    return dataset

<IPython.core.display.Javascript object>

### Data preprocessing

In [5]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [6]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [8]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return model, score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

In [9]:
def evaluate_model(model, dataset):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [10]:
def custom_cross_validate(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    cement_types=None,
    estimator_params=None,
    imputer_params=None,
    split_by_cement_type=True,
):
    results = []
    scores = []

    for train_index, test_index in cv.split(x):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "cement_types_train": cement_types.loc[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "cement_types_test": cement_types.loc[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        scores.append(score)

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]
    results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [11]:
class TransformerWrapper(RegressorMixin):
    def __init__(self, params):
        self.params = params
        self.model = self.get_model()
        self.batch_size = self.params.get("batch_size", 16)
        self.epochs = self.params.get("epochs", 300)
        self.verbose = self.params.get("verbose", 0)
        self.callbacks = self.params.get("callbacks", None)
        self.validation_split = self.params.get("validation_split", 0.0)
        self.num_features = self.params.get("num_features", x.shape[-1])

    def fit(self, X=None, y=None):
        self.history = self.model.fit(
            X,
            y,
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_split=self.validation_split,
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = Transformer(
            num_hid=self.params["num_features"],
            time_steps=self.params["timesteps"],
            num_head=self.params["num_heads"],
            num_layers_enc=self.params["num_layers_enc"],
            num_feed_forward=self.params["num_feed_forward"],
            dropout_rate=self.params.get("dropout_rate", 0.1),
            activation=self.params.get("activation", "relu"),
        )
        model.compile(
            tf.keras.optimizers.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [12]:
def pad_time_series(dataframe, timesteps):
    """
    Pad timeseries with zeros
    """
    df_tmp = pd.DataFrame(
        dict(
            zip(
                dataframe.columns,
                [[0 for _ in range(timesteps - 1)] for _ in range(dataframe.shape[1])],
            )
        )
    )
    df_tmp[DATE] = dataframe[DATE].iloc[0]
    return pd.concat([df_tmp, dataframe], axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [13]:
def split_sequences_per_cement_type(dataframe, timesteps, pad=False):
    """
    Create sequences per cement time
    to avoid having parts of the sequence
    of different types of cement.
    """
    if timesteps == 1:
        return split_sequences(
            dataframe.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps
        )

    dates = dataframe[DATE][timesteps - 1 :]
    data = []
    dataframes = []

    for cement_type in CEMENT_TYPES:
        data.append(dataframe[dataframe[cement_type] == 1])
    data.append(dataframe[(dataframe[CEMENT_TYPES] == 0).all(axis=1)])

    for df in data:
        if pad:
            dates = df[DATE].reset_index(drop=True)
            df = pad_time_series(df, timesteps).reset_index(drop=True)
        else:
            dates = df[DATE][timesteps - 1 :].reset_index(drop=True)
        x, y = split_sequences(df.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps)
        x = pd.DataFrame({"Sequences": [sample.tolist() for sample in x]})
        y = pd.DataFrame({"Target": y})
        dataframes.append(pd.concat([dates, x, y], axis=1))

    data = pd.concat(dataframes, axis=0)
    data[DATE] = pd.to_datetime(data[DATE])
    data = data.sort_values(by=DATE).reset_index(drop=True)
    x = data["Sequences"]
    y = data["Target"].values
    x = np.array(x.tolist())

    return x, y

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [14]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [15]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [16]:
index_to_save = 2
model_index = 1

<IPython.core.display.Javascript object>

In [17]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"
CEMENT_TYPES = [
    "Cement_Type_CEM B",
    "Cement_Type_CEM C",
    "Cement_Type_CP II-F-32",
    "Cement_Type_CP II-F-40",
    "Cement_Type_CP V-ARI",
    "Cement_Type_Type I-II",
    "Cement_Type_Type III",
    "Cement_Type_Type IL",
]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [18]:
results_to_save = []

results_dict = {
    "Category": "Global Model",
    "Company": "INN",
    "Plant": "INN",
    "Features": "Chemical + Mineralogical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Transformer",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [19]:
df = pd.read_csv("../../../../../../../data/processed/inn/global_dataset_inn.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

In [20]:
df_copy = df.copy()
df_copy = pd.get_dummies(data=df_copy, columns=["Cement_Type"], drop_first=True)

df_copy = df_copy.drop(
    [
        # "Cement_Type",
        # "Factory_Plant",
        "Blaine",
        # "#200",
        "#325",
        # "Final setting time",
        "Initial setting time",
        "CS1",
        "CS3",
        "CS7",
        # "Factory_Plant",
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

In [21]:
df_copy[CEMENT_TYPES] = df_copy[CEMENT_TYPES].astype(int)
dates = df["Date"].copy()
x = df_copy.drop(["Date", "CS28", "Factory_Plant"] + CEMENT_TYPES, axis=1)
y = df_copy["CS28"]
train_columns = x.columns

<IPython.core.display.Javascript object>

<h2>1. Dataset: df_copy</h2> <br>In this dataset all features are used.

In [22]:
def prepare_dataset(
    dataframe_original=df,
    dataframe_copy=df_copy,
    train_size=0.8,
    test_size=0.2,
    ignore_test_set=False,
    timesteps=1,
    split_by_cement_type=True,
):
    dataframe_original = df.copy()
    dataframe_copy = df_copy.copy()
    dataframe_copy[CEMENT_TYPES] = dataframe_copy[CEMENT_TYPES].astype(int).copy()
    dates = dataframe_original["Date"].copy()
    x = dataframe_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
    y = dataframe_copy["CS28"]
    cement_types = dataframe_copy[CEMENT_TYPES].copy()

    # Split the dataframe by plant
    plants = dataframe_original["Factory_Plant"].unique()
    train_indexes = []
    test_indexes = []
    plants_test_indexes = {}

    for plant in plants:
        plant_df = dataframe_original[dataframe_original["Factory_Plant"] == plant]
        plant_df = plant_df.sort_values(by="Date")
        plant_indices = plant_df.index
        train_end_index = int(len(plant_indices) * train_size)
        # plants_indexes[]

        if not ignore_test_set:
            train_indexes.extend(plant_indices[:train_end_index])
            test_indexes.extend(plant_indices[train_end_index:])
            plants_test_indexes[plant] = plant_indices[train_end_index:]

        else:
            train_indexes.extend(plant_indices[:train_end_index])
            test_indexes.extend(plant_indices[train_end_index:])
            plants_test_indexes[plant] = plant_indices[train_end_index:]

    train_index = dataframe_original.loc[train_indexes].sort_values(by="Date").index
    test_index = dataframe_original.loc[test_indexes].sort_values(by="Date").index

    plant_id_series = x["Factory_Plant"].copy()
    x = x.drop(["Factory_Plant"], axis=1)

    dataset = {
        "dates_train": dates[train_index].reset_index(drop=True).copy(),
        "cement_types_train": cement_types.loc[train_index]
        .reset_index(drop=True)
        .copy(),
        "x_train": x.loc[train_index].reset_index(drop=True).copy(),
        "y_train": y[train_index].reset_index(drop=True).copy(),
        "dates_test": dates[test_index].reset_index(drop=True).copy(),
        "cement_types_test": cement_types.loc[test_index].reset_index(drop=True).copy(),
        "x_test": x.loc[test_index].reset_index(drop=True).copy(),
        "y_test": y[test_index].reset_index(drop=True).copy(),
    }

    # Preprocess the dataset
    dataset = preprocess_data(dataset, None, SimpleImputer, {"strategy": "median"})
    # return dataset, plant_id_series.loc[test_index].reset_index(drop=True).copy()
    dataset["x_test"] = np.concatenate(
        [
            dataset["x_test"],
            plant_id_series.loc[test_index]
            .reset_index(drop=True)
            .copy()
            .values.reshape(-1, 1),
        ],
        axis=1,
    )
    index_list = x.columns.tolist()
    index_list.append("Factory_Plant")
    test_columns = pd.Index(index_list)

    # generate sequences (3D format)
    dataset = generate_sequences(
        dataset,
        timesteps=timesteps,
        split_by_cement_type=split_by_cement_type,
        train_columns=train_columns,
        test_columns=test_columns,
    )

    return dataset

<IPython.core.display.Javascript object>

In [23]:
def prepare_dataset_helper(timesteps):
    dataset = prepare_dataset(timesteps=timesteps)
    # Remove the last column (plant identification) for overall evaluation
    x_test_overall = np.delete(dataset["x_test"], -1, axis=2)

    # Initialize dictionaries to hold plant-specific datasets
    plant_specific_x_test = {}
    plant_specific_y_test = {}

    # Get unique plant identifiers
    unique_plants = np.unique(dataset["x_test"][:, 0, -1])

    for plant in unique_plants:
        # Create a mask for the current plant
        plant_mask = dataset["x_test"][:, 0, -1] == plant
        # Filter x_test and y_test using the mask
        x_test_plant = dataset["x_test"][plant_mask]
        y_test_plant = dataset["y_test"][plant_mask]

        # Remove the last column (plant identification) from x_test_plant
        x_test_plant = np.delete(x_test_plant, -1, axis=2)

        # Store the filtered arrays in the dictionaries
        plant_specific_x_test[plant] = x_test_plant
        plant_specific_y_test[plant] = y_test_plant

    # Output the shapes to verify
    # print("Overall x_test shape:", x_test_overall.shape)
    # for plant in plant_specific_x_test:
    #    print(f"Plant {plant} x_test shape:", plant_specific_x_test[plant].shape)
    #    print(f"Plant {plant} y_test shape:", plant_specific_y_test[plant].shape)

    plant_specific = {"x_test": plant_specific_x_test, "y_test": plant_specific_y_test}
    x_test_overall = x_test_overall.astype(float)
    dataset["x_test"] = x_test_overall
    return dataset, plant_specific

<IPython.core.display.Javascript object>

In [24]:
def print_scores_2(scores, METRICS, METRICS_DICT):
    for phase in ["test"]:
        print("******")
        print(f"[{phase.upper()}]")
        print("******")
        for metric in METRICS:
            name = METRICS_DICT[metric]
            print(
                f"{name}: %.3f (%.3f)"
                % (
                    np.mean(scores[f"{phase}_" + metric]),
                    np.std(scores[f"{phase}_" + metric]),
                )
            )
        print("\n======================\n")

<IPython.core.display.Javascript object>

In [25]:
def append_results(scores_dict):
    for plant, scores in scores_dict.items():
        results_dict_copy = results_dict.copy()
        results_dict_copy["Plant"] = plant
        results_dict_copy["Timesteps"] = timesteps
        results_dict_copy["Cross Validation"] = "Out of time"
        results_dict_copy[
            "Cross Validation Params"
        ] = '{"train_size": 0.8, "test_size": 0.2}'
        results_dict_copy["Data Shape"] = x.shape
        results_dict_copy["Model"] = f"Transformer_{model_index}"
        scores = {key: [value] for key, value in scores.items()}
        df_results = fill_results_dict(results_dict_copy, scores)
        results_to_save.append(df_results)


<IPython.core.display.Javascript object>

In [26]:
def print_results_and_get_scores_dict(scores, plant_specific):
    print("=======================")
    print("Overall Score")
    print("=======================\n")

    print_scores(scores, METRICS, METRICS_DICT)

    print("=======================")
    print("Scores per plant")
    print("=======================\n")

    scores_dict = {"INN": scores}

    for plant in df_copy["Factory_Plant"].unique():
        x_test = plant_specific["x_test"][plant].astype(float)
        y_test = plant_specific["y_test"][plant].astype(float)
        dataset["x_test"] = x_test
        dataset["y_test"] = y_test
        print("=======================")
        print(
            "Plant",
            plant,
        )
        print("=======================\n")
        scores_plant = evaluate_model(model, dataset)
        scores_dict[plant] = scores_plant
        print_scores_2(scores_plant, METRICS, METRICS_DICT)
    return scores_dict

<IPython.core.display.Javascript object>

In [27]:
def get_transformer_params(
    timesteps=1,
    num_features=x.shape[-1],
    num_heads=1,
    num_layers_enc=1,
    num_feed_forward=16,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=None,
    validation_split=0.0,
    verbose=0,
):
    params = {}
    params["num_features"] = x.shape[-1]
    params["timesteps"] = timesteps
    params["num_heads"] = num_heads
    params["num_layers_enc"] = num_layers_enc
    params["num_feed_forward"] = num_feed_forward
    params["verbose"] = verbose
    params["dropout_rate"] = dropout_rate
    params["activation"] = activation
    params["batch_size"] = batch_size
    params["epochs"] = epochs
    params["callbacks"] = callbacks
    params["validation_split"] = validation_split

    return params

<IPython.core.display.Javascript object>

In [28]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# Transformer Encoder

### Transformer 1

1. TIMESTEPS: 1

In [29]:
set_seeds()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=5,
    num_layers_enc=1,
    num_feed_forward=32,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

2024-07-15 23:57:40.789247: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-15 23:57:40.789293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2024-07-15 23:57:40.789300: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2024-07-15 23:57:40.789474: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.183.1
2024-07-15 23:57:40.789502: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.183.1
2024-07-15 23:57:40.789508: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.183.1


Minutes Elapsed:  0.5107662955919902


<IPython.core.display.Javascript object>

In [30]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 4.044 (0.000)
MAE: 3.330 (0.000)
MAPE: 0.077 (0.000)
R2: 0.502 (0.000)


******
[TEST]
******
RMSE: 5.036 (0.000)
MAE: 4.320 (0.000)
MAPE: 0.104 (0.000)
R2: 0.265 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.172 (0.000)
MAE: 3.232 (0.000)
MAPE: 0.066 (0.000)
R2: 0.538 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 6.156 (0.000)
MAE: 5.580 (0.000)
MAPE: 0.145 (0.000)
R2: -3.746 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.525 (0.000)
MAE: 4.043 (0.000)
MAPE: 0.097 (0.000)
R2: 0.176 (0.000)




<IPython.core.display.Javascript object>

### Transformer 1

1. TIMESTEPS: 7

In [31]:
set_seeds()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=5,
    num_layers_enc=1,
    num_feed_forward=32,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.956999909877777


<IPython.core.display.Javascript object>

In [32]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 2.533 (0.000)
MAE: 1.913 (0.000)
MAPE: 0.042 (0.000)
R2: 0.805 (0.000)


******
[TEST]
******
RMSE: 3.709 (0.000)
MAE: 2.958 (0.000)
MAPE: 0.069 (0.000)
R2: 0.599 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 3.033 (0.000)
MAE: 2.400 (0.000)
MAPE: 0.050 (0.000)
R2: 0.741 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.136 (0.000)
MAE: 3.578 (0.000)
MAPE: 0.092 (0.000)
R2: -1.289 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 3.769 (0.000)
MAE: 2.834 (0.000)
MAPE: 0.063 (0.000)
R2: 0.420 (0.000)




<IPython.core.display.Javascript object>

### Transformer 1

1. TIMESTEPS: 14

In [33]:
set_seeds()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=5,
    num_layers_enc=1,
    num_feed_forward=32,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.8756174286206563


<IPython.core.display.Javascript object>

In [34]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 2.467 (0.000)
MAE: 1.878 (0.000)
MAPE: 0.042 (0.000)
R2: 0.814 (0.000)


******
[TEST]
******
RMSE: 3.666 (0.000)
MAE: 2.826 (0.000)
MAPE: 0.068 (0.000)
R2: 0.604 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 2.509 (0.000)
MAE: 2.061 (0.000)
MAPE: 0.043 (0.000)
R2: 0.824 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 5.305 (0.000)
MAE: 4.649 (0.000)
MAPE: 0.120 (0.000)
R2: -2.912 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 2.473 (0.000)
MAE: 1.835 (0.000)
MAPE: 0.042 (0.000)
R2: 0.748 (0.000)




<IPython.core.display.Javascript object>

### Transformer 2

1. TIMESTEPS: 1

In [35]:
set_seeds()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=5,
    num_layers_enc=1,
    num_feed_forward=128,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.49105513095855713


<IPython.core.display.Javascript object>

In [36]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 4.009 (0.000)
MAE: 3.292 (0.000)
MAPE: 0.076 (0.000)
R2: 0.511 (0.000)


******
[TEST]
******
RMSE: 4.971 (0.000)
MAE: 4.266 (0.000)
MAPE: 0.102 (0.000)
R2: 0.284 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.619 (0.000)
MAE: 3.450 (0.000)
MAPE: 0.069 (0.000)
R2: 0.434 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 5.689 (0.000)
MAE: 5.145 (0.000)
MAPE: 0.133 (0.000)
R2: -3.052 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.541 (0.000)
MAE: 4.114 (0.000)
MAPE: 0.098 (0.000)
R2: 0.169 (0.000)




<IPython.core.display.Javascript object>

### Transformer 2

1. TIMESTEPS: 7

In [37]:
set_seeds()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=5,
    num_layers_enc=1,
    num_feed_forward=128,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.678760556379954


<IPython.core.display.Javascript object>

In [38]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 2.493 (0.000)
MAE: 1.912 (0.000)
MAPE: 0.042 (0.000)
R2: 0.811 (0.000)


******
[TEST]
******
RMSE: 3.895 (0.000)
MAE: 2.999 (0.000)
MAPE: 0.070 (0.000)
R2: 0.558 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 2.725 (0.000)
MAE: 2.190 (0.000)
MAPE: 0.046 (0.000)
R2: 0.791 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.167 (0.000)
MAE: 3.574 (0.000)
MAPE: 0.092 (0.000)
R2: -1.323 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.349 (0.000)
MAE: 3.096 (0.000)
MAPE: 0.069 (0.000)
R2: 0.228 (0.000)




<IPython.core.display.Javascript object>

### Transformer 2

1. TIMESTEPS: 14

In [39]:
set_seeds()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=5,
    num_layers_enc=1,
    num_feed_forward=128,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.6454097270965576


<IPython.core.display.Javascript object>

In [40]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.162 (0.000)
MAE: 2.410 (0.000)
MAPE: 0.053 (0.000)
R2: 0.695 (0.000)


******
[TEST]
******
RMSE: 4.636 (0.000)
MAE: 3.598 (0.000)
MAPE: 0.081 (0.000)
R2: 0.366 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.243 (0.000)
MAE: 3.123 (0.000)
MAPE: 0.065 (0.000)
R2: 0.498 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.139 (0.000)
MAE: 2.619 (0.000)
MAPE: 0.068 (0.000)
R2: -0.370 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 5.800 (0.000)
MAE: 4.745 (0.000)
MAPE: 0.104 (0.000)
R2: -0.387 (0.000)




<IPython.core.display.Javascript object>

### Transformer 3

1. TIMESTEPS: 1

In [41]:
set_seeds()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)


params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=3,
    num_layers_enc=4,
    num_feed_forward=256,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.1659379879633585


<IPython.core.display.Javascript object>

In [42]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 4.415 (0.000)
MAE: 3.546 (0.000)
MAPE: 0.083 (0.000)
R2: 0.407 (0.000)


******
[TEST]
******
RMSE: 5.260 (0.000)
MAE: 4.461 (0.000)
MAPE: 0.107 (0.000)
R2: 0.199 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 5.025 (0.000)
MAE: 3.901 (0.000)
MAPE: 0.080 (0.000)
R2: 0.330 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 5.730 (0.000)
MAE: 5.175 (0.000)
MAPE: 0.134 (0.000)
R2: -3.111 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 5.003 (0.000)
MAE: 4.263 (0.000)
MAPE: 0.105 (0.000)
R2: -0.008 (0.000)




<IPython.core.display.Javascript object>

### Transformer 3

1. TIMESTEPS: 7

In [43]:
set_seeds()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=3,
    num_layers_enc=4,
    num_feed_forward=256,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  2.1347837249437966


<IPython.core.display.Javascript object>

In [44]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 2.576 (0.000)
MAE: 1.882 (0.000)
MAPE: 0.041 (0.000)
R2: 0.798 (0.000)


******
[TEST]
******
RMSE: 3.917 (0.000)
MAE: 2.982 (0.000)
MAPE: 0.073 (0.000)
R2: 0.553 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 2.967 (0.000)
MAE: 2.366 (0.000)
MAPE: 0.049 (0.000)
R2: 0.753 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.790 (0.000)
MAE: 3.266 (0.000)
MAPE: 0.085 (0.000)
R2: -0.922 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.571 (0.000)
MAE: 3.185 (0.000)
MAPE: 0.080 (0.000)
R2: 0.147 (0.000)




<IPython.core.display.Javascript object>

### Transformer 3

1. TIMESTEPS: 14

In [45]:
set_seeds()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=3,
    num_layers_enc=4,
    num_feed_forward=256,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.5879924933115641


<IPython.core.display.Javascript object>

In [46]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 4.575 (0.000)
MAE: 3.454 (0.000)
MAPE: 0.076 (0.000)
R2: 0.362 (0.000)


******
[TEST]
******
RMSE: 4.467 (0.000)
MAE: 3.496 (0.000)
MAPE: 0.082 (0.000)
R2: 0.412 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 5.992 (0.000)
MAE: 5.009 (0.000)
MAPE: 0.111 (0.000)
R2: -0.001 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.134 (0.000)
MAE: 2.375 (0.000)
MAPE: 0.061 (0.000)
R2: -0.365 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.164 (0.000)
MAE: 3.383 (0.000)
MAPE: 0.079 (0.000)
R2: 0.285 (0.000)




<IPython.core.display.Javascript object>

### Transformer 4

1. TIMESTEPS: 1

In [47]:
set_seeds()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=4,
    num_layers_enc=2,
    num_feed_forward=128,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.8676807165145874


<IPython.core.display.Javascript object>

In [48]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.922 (0.000)
MAE: 2.970 (0.000)
MAPE: 0.070 (0.000)
R2: 0.532 (0.000)


******
[TEST]
******
RMSE: 4.553 (0.000)
MAE: 3.642 (0.000)
MAPE: 0.089 (0.000)
R2: 0.400 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.066 (0.000)
MAE: 3.148 (0.000)
MAPE: 0.065 (0.000)
R2: 0.562 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 5.418 (0.000)
MAE: 4.917 (0.000)
MAPE: 0.127 (0.000)
R2: -2.676 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.051 (0.000)
MAE: 2.912 (0.000)
MAPE: 0.073 (0.000)
R2: 0.339 (0.000)




<IPython.core.display.Javascript object>

### Transformer 4

1. TIMESTEPS: 7

In [49]:
set_seeds()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=4,
    num_layers_enc=2,
    num_feed_forward=128,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.8419479489326477


<IPython.core.display.Javascript object>

In [50]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.243 (0.000)
MAE: 2.574 (0.000)
MAPE: 0.056 (0.000)
R2: 0.680 (0.000)


******
[TEST]
******
RMSE: 3.873 (0.000)
MAE: 3.231 (0.000)
MAPE: 0.075 (0.000)
R2: 0.563 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 2.212 (0.000)
MAE: 1.800 (0.000)
MAPE: 0.037 (0.000)
R2: 0.862 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.804 (0.000)
MAE: 3.284 (0.000)
MAPE: 0.086 (0.000)
R2: -0.936 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.770 (0.000)
MAE: 4.219 (0.000)
MAPE: 0.094 (0.000)
R2: 0.071 (0.000)




<IPython.core.display.Javascript object>

### Transformer 4

1. TIMESTEPS: 14

In [51]:
set_seeds()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)


params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=4,
    num_layers_enc=2,
    num_feed_forward=128,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.1444973508516947


<IPython.core.display.Javascript object>

In [52]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.026 (0.000)
MAE: 2.288 (0.000)
MAPE: 0.050 (0.000)
R2: 0.721 (0.000)


******
[TEST]
******
RMSE: 4.198 (0.000)
MAE: 3.247 (0.000)
MAPE: 0.074 (0.000)
R2: 0.480 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 2.939 (0.000)
MAE: 2.393 (0.000)
MAPE: 0.050 (0.000)
R2: 0.759 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.224 (0.000)
MAE: 2.710 (0.000)
MAPE: 0.071 (0.000)
R2: -0.445 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 5.468 (0.000)
MAE: 4.288 (0.000)
MAPE: 0.092 (0.000)
R2: -0.232 (0.000)




<IPython.core.display.Javascript object>

### Transformer 5

1. TIMESTEPS: 1

In [53]:
set_seeds()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=7,
    num_layers_enc=2,
    num_feed_forward=64,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.7379505117734273


<IPython.core.display.Javascript object>

In [54]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 3.861 (0.000)
MAE: 3.100 (0.000)
MAPE: 0.069 (0.000)
R2: 0.546 (0.000)


******
[TEST]
******
RMSE: 4.463 (0.000)
MAE: 3.717 (0.000)
MAPE: 0.086 (0.000)
R2: 0.423 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.521 (0.000)
MAE: 3.296 (0.000)
MAPE: 0.066 (0.000)
R2: 0.458 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.560 (0.000)
MAE: 3.990 (0.000)
MAPE: 0.104 (0.000)
R2: -1.603 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.333 (0.000)
MAE: 3.793 (0.000)
MAPE: 0.086 (0.000)
R2: 0.244 (0.000)




<IPython.core.display.Javascript object>

### Transformer 5

1. TIMESTEPS: 7

In [55]:
set_seeds()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=7,
    num_layers_enc=2,
    num_feed_forward=64,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.492448914051056


<IPython.core.display.Javascript object>

In [56]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 2.364 (0.000)
MAE: 1.768 (0.000)
MAPE: 0.039 (0.000)
R2: 0.830 (0.000)


******
[TEST]
******
RMSE: 3.287 (0.000)
MAE: 2.565 (0.000)
MAPE: 0.062 (0.000)
R2: 0.685 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 2.538 (0.000)
MAE: 2.008 (0.000)
MAPE: 0.041 (0.000)
R2: 0.819 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.523 (0.000)
MAE: 4.006 (0.000)
MAPE: 0.104 (0.000)
R2: -1.737 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 2.398 (0.000)
MAE: 1.743 (0.000)
MAPE: 0.041 (0.000)
R2: 0.765 (0.000)




<IPython.core.display.Javascript object>

### Transformer 5

1. TIMESTEPS: 14

In [57]:
set_seeds()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)

params = get_transformer_params(
    timesteps=timesteps,
    num_features=x.shape[-1],
    num_heads=7,
    num_layers_enc=2,
    num_feed_forward=64,
    dropout_rate=0.1,
    activation="relu",
    batch_size=64,
    epochs=300,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    TransformerWrapper, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.4786046067873637


<IPython.core.display.Javascript object>

In [58]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 2.630 (0.000)
MAE: 1.952 (0.000)
MAPE: 0.042 (0.000)
R2: 0.789 (0.000)


******
[TEST]
******
RMSE: 3.004 (0.000)
MAE: 2.262 (0.000)
MAPE: 0.053 (0.000)
R2: 0.734 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 2.917 (0.000)
MAE: 2.288 (0.000)
MAPE: 0.047 (0.000)
R2: 0.763 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.267 (0.000)
MAE: 2.665 (0.000)
MAPE: 0.070 (0.000)
R2: -0.484 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 2.830 (0.000)
MAE: 1.909 (0.000)
MAPE: 0.044 (0.000)
R2: 0.670 (0.000)




<IPython.core.display.Javascript object>

# Saving the results

In [59]:
path = f"../../../../../../../reports/results/global_models/inn/all_cements/pre_training/full/"
filename = f"transformer_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [60]:
from sklearn.preprocessing import StandardScaler

<IPython.core.display.Javascript object>

In [61]:
columns_to_standardize = ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]

<IPython.core.display.Javascript object>

In [62]:
ddf = pd.concat(results_to_save).reset_index(drop=True)
ddf_copy = ddf.copy()
ddf_copy = ddf_copy[ddf_copy["Plant"].eq("INN")]
scaler = StandardScaler()
dddf_copy = ddf_copy.copy()
dddf_copy[columns_to_standardize] = scaler.fit_transform(
    ddf_copy[columns_to_standardize]
).copy()

ddf_copy["SCPM"] = (
    dddf_copy[["RMSE Test", "MAE Test", "MAPE Test"]].sum(axis=1) - dddf_copy["R2 Test"]
)
min_row = ddf_copy[ddf_copy["SCPM"].eq(ddf_copy["SCPM"].min())]
ddf.merge(min_row[["Timesteps", "Model"]], on=("Timesteps", "Model"), how="inner")

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,Cross Validation,Cross Validation Params,RMSE Train,MAE Train,MAPE Train,R2 Train,RMSE Test,MAE Test,MAPE Test,R2 Test
0,Global Model,INN,INN,Chemical + Mineralogical,"(3180, 13)",14,Transformer_15,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",2.629545,1.951607,0.042281,0.789125,3.004267,2.262451,0.053289,0.733954
1,Global Model,INN,partner_iv,Chemical + Mineralogical,"(3180, 13)",14,Transformer_15,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",2.629545,1.951607,0.042281,0.789125,2.916844,2.287506,0.046737,0.762712
2,Global Model,INN,partner_ii,Chemical + Mineralogical,"(3180, 13)",14,Transformer_15,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",2.629545,1.951607,0.042281,0.789125,3.267110,2.665097,0.069807,-0.483667
3,Global Model,INN,partner_i,Chemical + Mineralogical,"(3180, 13)",14,Transformer_15,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",2.629545,1.951607,0.042281,0.789125,2.829952,1.908867,0.044045,0.669901


<IPython.core.display.Javascript object>